In [1]:
%matplotlib notebook
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# We can view all of the classes that automap found
Base.classes.keys()

[]

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
query = session.query(Measurement.date, func.sum(Measurement.prcp)). \
group_by(Measurement.date).order_by(Measurement.id.desc()).limit(365).all()

query

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
data = pd.DataFrame(query, columns=['Date', 'Precipitation'])
data = data.sort_index(ascending=False)
data = data.reset_index()
data = data.drop("index", 1)
data = data.set_index('Date')
data.head()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
data.plot(figsize = (7.5, 7.5), rot = 45, use_index = True, legend=False)
# plt.tight_layout()
plt.ylabel('Precipation (inches)')
plt.xlabel('Date')
plt.title("Hawaii Precipitation: %s to %s" % (data.index.min(),data.index.max()))
plt.savefig("Output/BP_PrecipitationData")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
data_summary = data.describe()
data_summary

In [ ]:
# How many stations are available in this dataset?
StationCount = session.query(Station.id).count()
print(f'There are {StationCount} stations.')

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
ActiveStations = session.query(Measurement.station,func.count(Measurement.station)). \
                 group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
        
ActiveStations

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
MostActiveStationId = "USC00519281"

Active_TempObs = session.query(Measurement.date, Measurement.tobs).order_by(Measurement.id.desc()). \
    filter(Measurement.station == MostActiveStationId).limit(365).all()
    
Active_TempObs

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
TempObs = pd.DataFrame(Active_TempObs, columns=["Date", "Temperature Observations"])
TempObs.head()

In [ ]:
plt.figure(figsize = (8, 8))
plt.hist(TempObs["Temperature Observations"], bins=12)
plt.grid
plt.ylabel("Frequency")
plt.xlabel("Temperature (F)")
plt.title("Temperature over 12 Months")
plt.savefig("Output/BP_TemperatureHistogram")
plt.show()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
class Temperature():
    def __init__(self, min, avg, max):
        self.min = min
        self.avg = avg
        self.max = max
        

def calc_temps(start_date, end_date):
    min = session.query(func.min(Measurement.tobs)).filter(Measurement.date.between(start_date, end_date)).scalar()
    print(f'Minimum temperature: {min}')
    avg = session.query(func.round(func.avg(Measurement.tobs))).filter(Measurement.date.between(start_date, end_date)).scalar()
    print(f'Average temperature: {avg}')
    max = session.query(func.max(Measurement.tobs)).filter(Measurement.date.between(start_date, end_date)).scalar()
    print(f'Maximum temperature: {max}')
    
    Results = [{"Minimum":min},{"Maximum":max},{"Average":avg}]
    Results = Temperature(min, avg, max)
    return Results 



In [ ]:
# Use your previous function `calc_temps` to calculate the tmin, tavg, and tmax 
# for your trip using the previous year's data for those same dates.
result = calc_temps('2012-02-28', '2012-03-05')
result.min, result.avg, result.max

In [ ]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for the y value
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
plt.figure(figsize=(8,8))
plt.title("Trip Average Temperature")
plt.ylabel("Temperature (F)")
plt.bar(1, result.avg, yerr = (result.max - result.min), tick_label = "")

# Save an image of the chart and print to screen
plt.savefig("Output/TemperatureBarGraph")
plt.show()

In [ ]:
# Calculate the rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
rainfall = session.query(Measurement.station, Station.name, Station.latitude, Station.longitude, \
                        func.sum(Measurement.prcp)).filter(Measurement.date.between('2011-07-01','2011-07-15')). \
                        group_by(Measurement.station).all()
rainfall

## Optional Challenge Assignment

In [ ]:
# Create a query that will calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()
    
daily_normals("01-01")

In [ ]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip

# Use the start and end date to create a range of dates

# Stip off the year and save a list of %m-%d strings

# Loop through the list of %m-%d strings and calculate the normals for each date


In [ ]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index


In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
